In [1]:
import tflite_runtime.interpreter as tflite
from enum import Enum
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import pickle as pk

In [2]:
class Feature(Enum):
    MAG_AVG_FIELD = 0
    BX_GSE = 1
    SIGMA_B = 2
    SIGMA_BX = 3
    SIGMA_BY = 4
    SIGMA_BZ = 5
    NA_NP = 6
    SIGMA_PHI_V = 7
    SIGMA_THETA_V = 8
    PROT_FLUX_1 = 11
    PROT_FLUX_2 = 12
    PROT_FLUX_30 = 13
    PROT_FLUX_60 = 14
    PC_N = 10
    KP = 9

# Order of features in the trained model
features_order = [f.value for f in Feature]

# Load the new downloaded data for the past 24hrs
with open('./omni2_cl5f6i8QEF.lst', 'r',) as f:
    data = []
    for line in f:
        data.append(line.split())
df = pd.DataFrame(np.array(data, dtype=float))
# Remove the date columns
df.drop([0,1,2], axis='columns', inplace=True)
# Rename the columns
df.columns = range(df.shape[1])
# Reorder the columns to match the model features order
df = df[features_order]

In [8]:
def forecast(data):
    # Load tflite model
    interpreter = tflite.Interpreter(model_path="./exports/model.tflite")
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Load the standard scaler
    scaler = pk.load(open("./exports/standard_scaler.pkl", "rb"))

    # Scale Kp values from Kp*10 to Kp
    df[df.columns[-1]] = df[df.columns[-1]] / 10
    # Scale the data except for kp
    df[df.columns[:-1]] = scaler.transform(df[df.columns[:-1]])
    # Reshape the data to match the model input
    data = np.expand_dims(df, axis=0)
    interpreter.set_tensor(input_details[0]["index"], data.astype(np.float32))
    interpreter.invoke()
    pred = interpreter.get_tensor(output_details[0]["index"]).squeeze()
    pred = pred.round()
    pred = np.clip(pred, 0, 9)

    # Check if any of the predictions are stormy
    # storm = (pred >= 5).any()
    
    return pred

In [9]:
pred = forecast(df)

/home/nour/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
